In [1]:
from italian_csv_type_prediction.dataframe_generators import SimpleDatasetGenerator
from italian_csv_type_prediction.models import TypePredictor
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
from multiprocessing import Pool, cpu_count
from sklearn.metrics import accuracy_score, balanced_accuracy_score

In [2]:
def dataset_generation(number:int):
    return SimpleDatasetGenerator(verbose=False).build(number)

def _dataset_generation(args):
    return dataset_generation(*args)

def parallel_dataset_generation(number:int, chunks:int, processes:int):
    processes = min(processes, number)
    with Pool(processes) as p:
        Xs, ys = list(zip(*tqdm(
            p.imap(_dataset_generation, (
                (number//chunks, )
                for _ in range(chunks)
            )),
            desc="Creating dataset",
            total=chunks,
            leave=False
        )))
    return np.vstack(Xs), np.concatenate(ys)

In [3]:
processes = cpu_count()//2
chunks = processes*6
x_train, y_train = parallel_dataset_generation(10000, chunks, processes)
x_test, y_test = parallel_dataset_generation(1000, chunks, processes)

/home/luca/anaconda3/lib/python3.7/site-packages/dateutil/parser/_parser.py:1206: UnknownTimezoneWarning: tzname MEDE identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)
/home/luca/anaconda3/lib/python3.7/site-packages/dateutil/parser/_parser.py:1206: UnknownTimezoneWarning: tzname SALE identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


In [4]:
model = TypePredictor()

model.fit(x_train, y_train)

In [5]:
y_pred = model._model.predict(x_test)
y_train_pred = model._model.predict(x_train)

In [6]:
accuracy_score(y_test, y_pred), balanced_accuracy_score(y_test, y_pred)

(0.9919787077265514, 0.9933124609111992)

In [7]:
accuracy_score(y_train, y_train_pred), balanced_accuracy_score(y_train, y_train_pred)

(0.9929930760545754, 0.9942322944775197)

In [8]:
X, y = SimpleDatasetGenerator().generate_simple_dataframe(max_rows=20)

In [9]:
y

,ItalianFiscalCode,CadastreCode,Document,Tax,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean,Name,Surname,SurnameName,NameSurname
0,ItalianVAT,CadastreCode,Document,Tax,Plate,Address,Error,ProvinceCode,Region,Municipality,...,String,EMail,PhoneNumber,Date,NaN,Boolean,Company,Company,Company,Company
1,ItalianFiscalCode,NaN,Document,Tax,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean,Name,Surname,NaN,NameSurname
2,ItalianVAT,CadastreCode,Document,Tax,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,String,EMail,Error,Date,BiologicalSex,Boolean,Company,Company,Company,Company
3,ItalianFiscalCode,CadastreCode,Document,Tax,Plate,Address,ItalianZIPCode,ProvinceCode,NaN,Municipality,...,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean,NaN,Surname,NaN,NameSurname
4,ItalianVAT,CadastreCode,Document,Tax,Plate,Address,NaN,Error,Region,Municipality,...,String,EMail,PhoneNumber,Error,BiologicalSex,Boolean,Company,Company,Company,Company
5,ItalianFiscalCode,CadastreCode,Document,Tax,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,String,EMail,PhoneNumber,Error,BiologicalSex,Error,Name,Surname,SurnameName,NameSurname
6,ItalianFiscalCode,CadastreCode,Document,Tax,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,String,EMail,PhoneNumber,NaN,BiologicalSex,Boolean,Name,Surname,SurnameName,NameSurname
7,ItalianFiscalCode,CadastreCode,Document,NaN,NaN,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean,Name,Surname,SurnameName,NameSurname
8,ItalianVAT,CadastreCode,Document,Tax,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean,Company,Error,Error,Company
9,ItalianVAT,CadastreCode,Error,Tax,Plate,Address,Error,ProvinceCode,NaN,Municipality,...,String,EMail,NaN,Date,BiologicalSex,Boolean,Company,Company,Company,Company


In [10]:
model.predict_dataframe(X)

,ItalianFiscalCode,CadastreCode,Document,Tax,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean,Name,Surname,SurnameName,NameSurname
0,ItalianVAT,CadastreCode,Document,Tax,Plate,Address,Error,ProvinceCode,Region,Municipality,...,String,EMail,PhoneNumber,Date,NaN,Boolean,Company,Company,Company,Company
1,ItalianFiscalCode,NaN,Document,Tax,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean,Name,Surname,NaN,NameSurname
2,ItalianVAT,CadastreCode,Document,Tax,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,String,EMail,Error,Date,BiologicalSex,Boolean,Company,Company,Company,Company
3,ItalianFiscalCode,CadastreCode,Document,Tax,Plate,Address,ItalianZIPCode,ProvinceCode,NaN,Municipality,...,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean,NaN,Surname,NaN,NameSurname
4,ItalianVAT,CadastreCode,Document,Tax,Plate,Address,NaN,Error,Region,Municipality,...,String,EMail,PhoneNumber,Error,BiologicalSex,Boolean,Company,Company,Company,Company
5,ItalianFiscalCode,CadastreCode,Document,Tax,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,String,EMail,PhoneNumber,Error,BiologicalSex,Error,Name,Surname,SurnameName,NameSurname
6,ItalianFiscalCode,CadastreCode,Document,Tax,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,String,EMail,PhoneNumber,NaN,BiologicalSex,Boolean,Name,Surname,SurnameName,NameSurname
7,ItalianFiscalCode,CadastreCode,Document,NaN,NaN,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean,Name,Surname,SurnameName,NameSurname
8,ItalianVAT,CadastreCode,Document,Tax,Plate,Address,ItalianZIPCode,ProvinceCode,Region,Municipality,...,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean,Company,Company,Company,Company
9,ItalianVAT,CadastreCode,Document,Tax,Plate,Address,Error,ProvinceCode,NaN,Municipality,...,String,EMail,NaN,Date,BiologicalSex,Boolean,Company,Company,Company,Company


In [11]:
from collections import Counter

mask = y_test != y_pred

true_labels = model._embedder._encoder.inverse_transform(y_test[mask])
predicted_labels = model._embedder._encoder.inverse_transform(y_pred[mask])

Counter(zip(true_labels, predicted_labels))

Counter({('Error', 'Document'): 638,
         ('Error', 'ItalianZIPCode'): 149,
         ('Error', 'Company'): 3109,
         ('Company', 'NaN'): 211,
         ('Error', 'ProvinceCode'): 23,
         ('Company', 'Error'): 442,
         ('NaN', 'Company'): 156,
         ('ItalianZIPCode', 'Error'): 68,
         ('Surname', 'Company'): 7,
         ('Integer', 'Float'): 3,
         ('Error', 'PhoneNumber'): 42,
         ('Float', 'Integer'): 78,
         ('NaN', 'CountryCode'): 51,
         ('Error', 'NaN'): 4,
         ('NameSurname', 'Error'): 3,
         ('Float', 'NaN'): 3,
         ('CountryCode', 'NaN'): 31,
         ('Municipality', 'Company'): 6,
         ('Error', 'CountryCode'): 23,
         ('Error', 'String'): 27,
         ('Company', 'String'): 46,
         ('NameSurname', 'String'): 11,
         ('Integer', 'NaN'): 2,
         ('Name', 'String'): 4,
         ('Surname', 'String'): 1,
         ('Error', 'Boolean'): 5,
         ('Error', 'Municipality'): 6,
         ('Surname'